In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:

from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib as mpl
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib import rcParams
#rcParams['figure.figsize'] = 20, 10
plt.style.use('ggplot')

### Intro

I am complete noob to basket ball. But that doesn't stop me from doing some analysis with it,especially when I recieve an email from kaggle annoucing the start of a competition.  If any basketball enthusiast is mad at me , please lets call truce.

The main goal of this competition is to make predictions for all possible matchups from the past four NCAA® tournaments (seasons 2014, 2015, 2016, and 2017). In Stage 2, we will be asked to make predictions for all possible matchups from the current NCAA® tournament (season 2018).

The main goal of this notebook however is to not predict anything. Its only geared towards exploratory data analysis, even simpler to make pretty(in my totally objective eyes) graphs.



### Alright lets start loading datasets

In [ ]:
players_2010=pd.read_csv("../input/Players_2010.csv")
players_2011=pd.read_csv("../input/Players_2011.csv")
players_2012=pd.read_csv("../input/Players_2012.csv")
players_2013=pd.read_csv("../input/Players_2013.csv")
players_2014=pd.read_csv("../input/Players_2014.csv")
players_2015=pd.read_csv("../input/Players_2015.csv")
players_2016=pd.read_csv("../input/Players_2016.csv")
players_2017=pd.read_csv("../input/Players_2017.csv")

Lets concatenate all the players files in one

In [ ]:
players=pd.concat([players_2010,players_2011,players_2012,players_2013,players_2014,players_2015,players_2016,players_2017])

In [ ]:
players.shape

In [ ]:
players.head()

In [ ]:
seeds = pd.read_csv('../input/NCAATourneySeeds.csv')
tourney_results = pd.read_csv('../input/NCAATourneyCompactResults.csv')
regular_results = pd.read_csv('../input/RegularSeasonCompactResults.csv')

Let's' take a look at this

In [ ]:
tourney_results.head()

Taking inspiration from "I didnt read rules kernel" Let me post here some details about Data

**Special note about "Season" numbers: the college basketball season lasts from early November until the national championship tournament that starts in the middle of March. For instance, this year the first men’s Division I games were played on November 10th, 2017 and the men’s national championship game will be played on April 2nd, 2018. Because a basketball season spans two calendar years like this, it can be confusing to refer to the year of the season. By convention, when we identify a particular season, we will reference the year that the season ends in, not the year that it starts in. So for instance, the current season will be identified in our data as the 2018 season, not the 2017 season or the 2017-18 season or the 2017-2018 season, though you may see any of these in everyday use outside of our data.**

### Plots
Enough with numbers lets see some graphs which we can take in easily

In [ ]:
tourney_results["W-LScore"]=tourney_results["WScore"]-tourney_results["LScore"]

In [ ]:
fig=plt.figure(figsize=(10,10))
ax1=fig.add_subplot(2, 2, 1)
tourney_results.groupby("Season")["WScore"].mean().plot(kind="bar",ax=ax1)
ax2=fig.add_subplot(2, 2, 2)
tourney_results.groupby("Season")["LScore"].mean().plot(kind="bar",ax=ax2)
ax3=fig.add_subplot(2, 2, 3)
tourney_results.groupby("Season")["W-LScore"].mean().plot(kind="bar",ax=ax3)
plt.tight_layout()

Violin plots can summarise the variations in their plots gracefully

In [ ]:
fig=plt.figure(figsize=(15,10))
(tourney_results
.pipe((sns.violinplot,"data"),x="Season",y="W-LScore"))

From this graph and below boxplot we can see that there is a significant different between years in win-loss score during tournament season.

In [ ]:
fig=plt.figure(figsize=(15,10))
(tourney_results
.pipe((sns.boxplot,"data"),x="Season",y="W-LScore"))

Lets check the same for regular season as well

In [ ]:
regular_results["W-LScore"]=regular_results["WScore"]-regular_results["LScore"]

In [ ]:
fig=plt.figure(figsize=(15,10))
(regular_results
.pipe((sns.boxplot,"data"),x="Season",y="W-LScore")
)

This graph is much more consistent across the seasons. Except in 1992.  The violinplot below shows the same.

In [ ]:
fig=plt.figure(figsize=(15,10))
(regular_results
.pipe((sns.violinplot,"data"),x="Season",y="W-LScore")
)

### Lets see which teams have the best W-L Score

In [ ]:
teams=pd.read_csv("../input/Teams.csv")

In [ ]:
len(teams)

Wow 364 different teams participated in the regular season over the years. Thats incredible.

Lets join teams with regular season results and tourney results.
let's drop some columns and rename others to get it in the shape we need.
If you know a quicker way to do this ,please mention it in comments, I would like to simplify this.

In [ ]:
regular_results_team=pd.merge(regular_results,teams,how="left",left_on="WTeamID",right_on="TeamID")
regular_results_team.drop(["TeamID","FirstD1Season","LastD1Season"],axis=1,inplace=True)
regular_results_team.rename(columns={"TeamName":"WTeamName"},inplace=True)

In [ ]:
regular_results_team=pd.merge(regular_results_team,teams,how="left",left_on="LTeamID",right_on="TeamID")
regular_results_team.drop(["TeamID","FirstD1Season","LastD1Season"],axis=1,inplace=True)
regular_results_team.rename(columns={"TeamName":"LTeamName"},inplace=True)

In [ ]:
regular_results_team.head()

In [ ]:
fig=plt.figure(figsize=(15,10))
(regular_results_team
.groupby("WTeamName",as_index=False)
["W-LScore"]
.mean()
.pipe((sns.stripplot,"data"),x="WTeamName",y="W-LScore") 
)

This graph doesnt tell much as there are way more teams than we can realistically see in a single plot
Lets filter them by good and bad teams

In [ ]:
team_mean_WLScore=(
regular_results_team
.groupby("WTeamName",as_index=False)
["W-LScore"]
.mean()
)

In [ ]:
team_mean_WLScore["quality"]=(
pd.cut(team_mean_WLScore["W-LScore"],bins= 5,labels=["bad", "medium1","medium2" ,"good","great"])
)

In [ ]:
team_mean_WLScore.head()

In [ ]:
fig=plt.figure(figsize=(20,8))
ax1=fig.add_subplot(1,2,1)
ax1=(team_mean_WLScore
.query("quality=='great'")
.pipe((sns.stripplot,"data"),x="WTeamName",y="W-LScore",ax=ax1) 
)
ax1.set_title("The Great")

ax2=fig.add_subplot(1,2,2)
ax2=(team_mean_WLScore
.query("quality=='bad'")
.pipe((sns.stripplot,"data"),x="WTeamName",y="W-LScore",ax=ax2) 
)
ax2.set_title("The bad")

The above graphs gives us the first clue about which teams are truly best vs 
which teams could be bad  in a regular season.  I say could be, because there could be some teams which always win by low margins, however still win a lot.  As I mentioned earlier my lack of knowledge in this domain means I have to do a very generic analysis. 

### To be continued